### this notebook is a copy of train.py 

In [1]:
from rl.callbacks import ModelIntervalCheckpoint, FileLogger
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Concatenate
from gym import spaces
import numpy as np
from PIL import Image
from configparser import ConfigParser
import os
from os.path import join, pardir, exists
from pathlib import Path

from gym_airsim.airsim_car_env import AirSimCarEnv

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

class AirSimCarProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        # We could perform this processing step in `process_observation`. In this case, however,
        # we would need to store a `float32` array instead, which is 4x more memory intensive than
        # an `uint8` array. This matters if we store 1M observations.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

config = ConfigParser()
config.read('config.ini')
num_actions = int(config['car_agent']['actions'])
                    
WINDOW_LENGTH = 4
INPUT_SHAPE = (84, 84)

env = AirSimCarEnv()
np.random.seed(123)

# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_actions))
model.add(Activation('linear'))
print(model.summary())

log_dir = 'logs'
if not exists(log_dir):
    os.makedirs(log_dir)
tensorboard_dir = Path("logs/keras-rl_dqn_AirsimCar_tensorboard/")
if not exists(tensorboard_dir):
    os.makedirs(tensorboard_dir)

memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),  attr='eps', value_max=1., 
                              value_min=.1, value_test=.05, nb_steps=1000000)
processor = AirSimCarProcessor()

dqn = DQNAgent(model=model, nb_actions=num_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.0001), metrics=['mae'])

def build_callbacks(env_name):
    log_dir = 'logs'
    if not exists(log_dir):
        os.makedirs(log_dir)
    
    checkpoint_weights_filename = join(log_dir, 'dqn_' + env_name + '_weights_{step}.h5f')
    log_filename = join(log_dir,'dqn_{}_log.json'.format(env_name))
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
    callbacks += [FileLogger(log_filename, interval=100)]
    return callbacks

callbacks = build_callbacks('AirSimCarRL')

dqn.fit(env, nb_steps=1000000,
        visualize=False,
        verbose=2,
        callbacks=callbacks)

Using TensorFlow backend.
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Hoang\Anaconda3\envs\tf_gpu\lib\sit

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)

['WayPoint0', 'WayPoint1', 'WayPoint2', 'WayPoint3', 'WayPoint4', 'WayPoint5', 'WayPoint6', 'WayPoint7', 'WayPoint8', 'WayPoint9']



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_2 (Activation)    (None, 9, 9, 64)          0         
_____________________________________________________

   3412/1000000: episode: 17, duration: 7.610s, episode steps: 179, steps per second: 24, episode reward: 64.400, mean reward: 0.360 [-1.000, 0.500], mean action: 2.883 [0.000, 6.000], mean observation: 172.089 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   3617/1000000: episode: 18, duration: 9.439s, episode steps: 205, steps per second: 22, episode reward: 130.800, mean reward: 0.638 [-1.000, 1.000], mean action: 3.034 [0.000, 6.000], mean observation: 171.188 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   3812/1000000: episode: 19, duration: 8.486s, episode steps: 195, steps per second: 23, episode reward: 78.800, mean reward: 0.404 [-1.000, 0.500], mean action: 2.969 [0.000, 6.000], mean observation: 172.394 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   3970/1000000: episode: 20, duration: 6.668s, episode steps: 158, steps per second: 24, episode reward: 57.500, mean reward: 0.364

   8997/1000000: episode: 46, duration: 7.159s, episode steps: 171, steps per second: 24, episode reward: 56.000, mean reward: 0.327 [-1.000, 0.500], mean action: 2.953 [0.000, 6.000], mean observation: 172.070 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   9198/1000000: episode: 47, duration: 8.885s, episode steps: 201, steps per second: 23, episode reward: 77.800, mean reward: 0.387 [-1.000, 0.500], mean action: 2.945 [0.000, 6.000], mean observation: 171.758 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   9423/1000000: episode: 48, duration: 10.598s, episode steps: 225, steps per second: 21, episode reward: 131.300, mean reward: 0.584 [-1.000, 1.000], mean action: 2.987 [0.000, 6.000], mean observation: 172.342 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
   9625/1000000: episode: 49, duration: 9.295s, episode steps: 202, steps per second: 22, episode reward: 113.300, mean reward: 0.5

  14823/1000000: episode: 75, duration: 9.301s, episode steps: 203, steps per second: 22, episode reward: 97.300, mean reward: 0.479 [-1.000, 1.000], mean action: 3.123 [0.000, 6.000], mean observation: 172.623 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
  15037/1000000: episode: 76, duration: 9.967s, episode steps: 214, steps per second: 21, episode reward: 162.300, mean reward: 0.758 [-1.000, 1.000], mean action: 3.121 [0.000, 6.000], mean observation: 172.435 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
  15215/1000000: episode: 77, duration: 7.508s, episode steps: 178, steps per second: 24, episode reward: 57.100, mean reward: 0.321 [-1.000, 0.500], mean action: 3.051 [0.000, 6.000], mean observation: 173.005 [0.000, 255.000], loss: --, mean_absolute_error: --, mean_q: --, mean_eps: --
  15412/1000000: episode: 78, duration: 8.666s, episode steps: 197, steps per second: 23, episode reward: 79.800, mean reward: 0.405

...

 744356/1000000: episode: 3213, duration: 20.450s, episode steps: 466, steps per second: 23, episode reward: 371.000, mean reward: 0.796 [0.500, 1.000], mean action: 3.655 [0.000, 6.000], mean observation: 165.919 [0.000, 255.000], loss: 0.362913, mean_absolute_error: 27.676424, mean_q: 32.398898, mean_eps: 0.330290
 744827/1000000: episode: 3214, duration: 20.469s, episode steps: 471, steps per second: 23, episode reward: 373.700, mean reward: 0.793 [0.100, 1.000], mean action: 3.652 [0.000, 6.000], mean observation: 165.835 [0.000, 255.000], loss: 0.371047, mean_absolute_error: 27.868144, mean_q: 32.628261, mean_eps: 0.329869
 745301/1000000: episode: 3215, duration: 20.472s, episode steps: 474, steps per second: 23, episode reward: 418.500, mean reward: 0.883 [0.500, 1.000], mean action: 3.540 [0.000, 6.000], mean observation: 165.315 [0.000, 255.000], loss: 0.375165, mean_absolute_error: 27.962522, mean_q: 32.768117, mean_eps: 0.329442
 745781/1000000: episode: 3216, duration: 21.1

 756058/1000000: episode: 3239, duration: 20.414s, episode steps: 466, steps per second: 23, episode reward: 412.500, mean reward: 0.885 [0.500, 1.000], mean action: 3.401 [0.000, 6.000], mean observation: 164.896 [0.000, 255.000], loss: 0.376772, mean_absolute_error: 28.218794, mean_q: 33.038893, mean_eps: 0.319758
 756530/1000000: episode: 3240, duration: 20.412s, episode steps: 472, steps per second: 23, episode reward: 405.500, mean reward: 0.859 [0.500, 1.000], mean action: 3.576 [0.000, 6.000], mean observation: 165.093 [0.000, 255.000], loss: 0.392486, mean_absolute_error: 28.352395, mean_q: 33.223162, mean_eps: 0.319335
 756906/1000000: episode: 3241, duration: 15.309s, episode steps: 376, steps per second: 25, episode reward: 303.900, mean reward: 0.808 [-1.000, 1.000], mean action: 3.644 [0.000, 6.000], mean observation: 172.997 [0.000, 255.000], loss: 0.413840, mean_absolute_error: 27.886355, mean_q: 32.664304, mean_eps: 0.318954
 757384/1000000: episode: 3242, duration: 20.

 767978/1000000: episode: 3265, duration: 20.558s, episode steps: 465, steps per second: 23, episode reward: 386.500, mean reward: 0.831 [0.500, 1.000], mean action: 3.677 [0.000, 6.000], mean observation: 165.817 [0.000, 255.000], loss: 0.383902, mean_absolute_error: 28.630064, mean_q: 33.508945, mean_eps: 0.309029
 768445/1000000: episode: 3266, duration: 20.413s, episode steps: 467, steps per second: 23, episode reward: 375.500, mean reward: 0.804 [0.500, 1.000], mean action: 3.600 [0.000, 6.000], mean observation: 165.148 [0.000, 255.000], loss: 0.402125, mean_absolute_error: 28.265387, mean_q: 33.100690, mean_eps: 0.308609
 768912/1000000: episode: 3267, duration: 20.473s, episode steps: 467, steps per second: 23, episode reward: 424.000, mean reward: 0.908 [0.500, 1.000], mean action: 3.610 [0.000, 6.000], mean observation: 165.687 [0.000, 255.000], loss: 0.383436, mean_absolute_error: 28.756914, mean_q: 33.674943, mean_eps: 0.308190
 769382/1000000: episode: 3268, duration: 20.5

 779958/1000000: episode: 3291, duration: 20.404s, episode steps: 468, steps per second: 23, episode reward: 376.000, mean reward: 0.803 [0.500, 1.000], mean action: 3.404 [0.000, 6.000], mean observation: 165.944 [0.000, 255.000], loss: 0.426707, mean_absolute_error: 29.100784, mean_q: 34.061735, mean_eps: 0.298248
 780432/1000000: episode: 3292, duration: 20.514s, episode steps: 474, steps per second: 23, episode reward: 378.500, mean reward: 0.799 [0.500, 1.000], mean action: 3.399 [0.000, 6.000], mean observation: 165.508 [0.000, 255.000], loss: 0.396091, mean_absolute_error: 29.213315, mean_q: 34.230978, mean_eps: 0.297825
 780902/1000000: episode: 3293, duration: 20.493s, episode steps: 470, steps per second: 23, episode reward: 421.000, mean reward: 0.896 [0.500, 1.000], mean action: 3.411 [0.000, 6.000], mean observation: 165.556 [0.000, 255.000], loss: 0.394577, mean_absolute_error: 29.170374, mean_q: 34.152006, mean_eps: 0.297401
 781366/1000000: episode: 3294, duration: 20.4

 791790/1000000: episode: 3317, duration: 20.484s, episode steps: 472, steps per second: 23, episode reward: 406.500, mean reward: 0.861 [0.500, 1.000], mean action: 3.441 [0.000, 6.000], mean observation: 165.714 [0.000, 255.000], loss: 0.373558, mean_absolute_error: 29.899232, mean_q: 35.017374, mean_eps: 0.287601
 792254/1000000: episode: 3318, duration: 20.463s, episode steps: 464, steps per second: 23, episode reward: 415.000, mean reward: 0.894 [0.500, 1.000], mean action: 3.403 [0.000, 6.000], mean observation: 165.520 [0.000, 255.000], loss: 0.399880, mean_absolute_error: 29.649220, mean_q: 34.721098, mean_eps: 0.287180
 792711/1000000: episode: 3319, duration: 20.562s, episode steps: 457, steps per second: 22, episode reward: 408.000, mean reward: 0.893 [0.500, 1.000], mean action: 3.567 [0.000, 6.000], mean observation: 165.401 [0.000, 255.000], loss: 0.409804, mean_absolute_error: 29.709831, mean_q: 34.819092, mean_eps: 0.286766
 793180/1000000: episode: 3320, duration: 20.4

### continue training 

In [2]:
# def build_callbacks_cont(env_name):
#     checkpoint_weights_filename = join(log_dir, 'dqn_' + env_name + '_weights_500000_add_{step}.h5f')
#     log_filename = join(log_dir,'dqn_{}_log.json'.format(env_name))
#     callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=25000)]
#     callbacks += [FileLogger(log_filename, interval=100)]
#     return callbacks

# callbacks_cont = build_callbacks_cont('AirSimCarRL')

# dqn = DQNAgent(model=model, nb_actions=num_actions, policy=policy, memory=memory,
#                processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
#                train_interval=4, delta_clip=1.)
# dqn.compile(Adam(lr=.0001), metrics=['mae'])

# weights_filename = join(log_dir,'dqn_{}_weights_{}.h5f'.format('AirSimCarRL', 500000))
# dqn.load_weights(weights_filename)

# dqn.fit(env, nb_steps=2000000,
#         visualize=False,
#         verbose=2,
#         callbacks=callbacks_cont)

### testing

In [ ]:
# log_dir = 'logs'
# if not exists(log_dir):
#     os.makedirs(log_dir)
    
# weights_filename = join(log_dir,'dqn_{}_weights_{}.h5f'.format('AirSimCarRL', 1000000))
# dqn.load_weights(weights_filename)
# dqn.test(env, nb_episodes=10, visualize=True)